###### Join Expressions:
* Join brings two datasets together by comparing by comparing the value of one or more keys of the left and right and evaluating the result of a join expression
* Join expression determines whether Spark should bring together the left set of data with the right set of data.
* an **equi-join expression**, compares whether the specified keys in your left and right datasets are equal.
* Spark also allows for much more sophsticated join policies in addition to equi-joins. We can even use **complex types and perform something like checking whether a key exists within an array when you perform a join.**

###### Join Types:
join type determines what should be in the result set. There are a variety of different join types available in Spark

* Inner Joins - Keep rows with keys that exist in the left and right datasets
* Outter Joins - Keep rows with keys in either the left or right datasets
* Left Outer Joins - Keep rows with keys in the left dataset
* Right Outer Joins - Keep rows with keys in the right dataset
* Left Semi Join - Keep the rows in the left, and only the left, dataset where the key appears in the right dataset.
* Left anti Join - keep the rows in the left, and only the left, dataset where they do not appear in the right dataset.
* Natural joins - Perform a join by implicitly matching the columns between the two datasets with the same names.
* Cross (or Cartesian) joins - Match every row in the left dataset with every row in the right dataset.

let’s create some simple datasets to understand more with examples.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('joins').getOrCreate()

person = spark.createDataFrame([
        (0, "Bill Chambers", 0, [100]),
        (1, "Matei Zaharia", 1, [500, 250, 100]),
        (2, "Michael Armbrust", 1, [250, 100])])\
        .toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
          (0, "Masters", "School of Information", "UC Berkeley"),
          (2, "Masters", "EECS", "UC Berkeley"),
          (1, "Ph.D.", "EECS", "UC Berkeley")])\
          .toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
              (500, "Vice President"),
              (250, "PMC Member"),
              (100, "Contributor")])\
              .toDF("id", "status")

22/10/19 11:59:07 WARN Utils: Your hostname, tars resolves to a loopback address: 127.0.1.1; using 192.168.1.66 instead (on interface wlan0)
22/10/19 11:59:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 11:59:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [4]:
person.show()
graduateProgram.show()
sparkStatus.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



###### Inner Joins:
Inner joins evaluate the keys in both of the DataFrames or tables and include (and join together) only the rows that evaluate to true.

Eg: we join the graduateProgram DataFrame with the person DataFrame to create a new DataFrame

In [5]:
joinExpression = person["graduate_program"] == graduateProgram["id"]

In [6]:
wrongJoinExpression = person["name"] == graduateProgram["school"]

Inner joins are the default join, so we just need to specify our left DataFrame and join the right in the JOIN expression:

In [7]:
person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [8]:
joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [9]:
%sql
SELECT * FROM person INNER JOIN graduateProgram ON person.graduate_program = graduateProgram.id

SyntaxError: invalid syntax (1277972674.py, line 2)

###### Outer Join:
* Outer joins evaluate the keys in both of the DataFrames or tables and includes (and joins together) the rows that evaluate to true or false. 
* If there is no equivalent row in either the left or right DataFrame, Spark will insert null

In [10]:
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



###### Left Outer Joins:
* Left outer joins evaluate the keys in both of the DataFrames or tables and includes all rows from the left DataFrame as well as any rows in the right DataFrame that have a match in the left DataFrame.
* If there is no equivalent row in the right DataFrame, Spark will insert null

In [12]:
joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
|  1|  Ph.D.|                EECS|UC Berkeley|   2|Michael Armbrust|               1|     [250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



###### Right Outer Joins:
* Right outer joins evaluate the keys in both of the DataFrames or tables and includes all rows from the right DataFrame as well as any rows in the left DataFrame that have a match in the right DataFrame.
* If there is no equivalent row in the left DataFrame, Spark will insert null

In [13]:
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



###### Left Semi Join:
* This join actually include doesnt any values from the right DataFrame.
* They only compare to see if value exists in second dataframe.
* If the value does exist, those rows will be kept in the result, even if there are duplicate keys in the left DataFrame.

In [14]:
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [15]:
#Adding duplicate row to check how the duplicate works

gradProgram2 = graduateProgram.union(spark.createDataFrame([
                      (0, "Masters", "Duplicated Row", "Duplicated School")]))

gradProgram2.createOrReplaceTempView("gradProgram2")
gradProgram2.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



###### Left Anti Joins:
* This join is exactly opposite to Left Semi Join.
* This also compare values to see if the value exists in the second DataFrame. However, rather than keeping the values that exist in the second DataFrame, they keep only the values that do not have a corresponding key in the second DataFrame.
* Think of anti joins as a NOT IN SQL-style filter

In [16]:
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



###### Cross (Cartesian) Joins:
* Cross joins will join every single row in the left DataFrame to ever single row in the right DataFrame.
* This will cause an absolute explosion in the number of rows contained in the resulting DataFrame.

In [19]:
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [18]:
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC 

* You should use cross-joins only if you are absolutely, 100 percent sure that this is the join you need.Reason being we need to be explicit when defining a cross-join in Spark.

* Advanced users can set the session-level configuration **spark.sql.crossJoin.enable** to true in order to allow cross-joins without warnings or without Spark trying to perform another join for you.

###### Challenges When Using Joins:
**Joining Complex Types:**
* Any expression is a valid join expression, assuming that it returns a Boolean

In [20]:
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId").join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



**Handling Duplicate Column Names:**

* In a DataFrame, each column has a unique ID within Spark’s SQL Engine,Catalyst. This unique ID is purely internal and not something that you can directly reference. 
* Duplicate column names occur in two distinct situations:
  * The Join expression that you specify does not remove one key from one of the input DataFrames and the keys have the same column name.
  * Two columns on which we are not performing the join have the same name..

In [21]:
person.show()
graduateProgram.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [22]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
joinExpr = gradProgramDupe["graduate_program"] == person["graduate_program"]

#joinExpression = person["graduate_program"] == graduateProgram["id"]

In [23]:
person.join(gradProgramDupe, joinExpr).show()

+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|graduate_program| degree|          department|     school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|               0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+



The challenge arises when we refer to one of these columns

In [24]:
person.join(gradProgramDupe, joinExpr).select("graduate_program").show()

AnalysisException: Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.

**Approach 1: Different join expression**
The easiest fix is to change the join expression from a Boolean expression to a string or sequence. This automatically removes one of
the columns for you during the join

In [25]:
person.join(gradProgramDupe,"graduate_program").show()

+----------------+---+----------------+---------------+-------+--------------------+-----------+
|graduate_program| id|            name|   spark_status| degree|          department|     school|
+----------------+---+----------------+---------------+-------+--------------------+-----------+
|               0|  0|   Bill Chambers|          [100]|Masters|School of Informa...|UC Berkeley|
|               1|  1|   Matei Zaharia|[500, 250, 100]|  Ph.D.|                EECS|UC Berkeley|
|               1|  2|Michael Armbrust|     [250, 100]|  Ph.D.|                EECS|UC Berkeley|
+----------------+---+----------------+---------------+-------+--------------------+-----------+



In [26]:
person.join(gradProgramDupe,"graduate_program").select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



**Approach 2: Dropping the column after the join**
* When doing this, we need to refer to the column via the original source DataFrame.
* We can do this if the join uses the same key names or if the source DataFrames have columns that simply have the same name

In [27]:
person.join(gradProgramDupe, joinExpr).drop(person["graduate_program"]).select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



This is an artifact of Spark’s SQL analysis process in which an explicitly referenced column will pass analysis because Spark has no need to resolve the column.

**Approach 3: Renaming a column before the join:**

We can avoid this issue altogether if we rename one of our columns before the join

In [28]:
gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
joinExpr = person["graduate_program"] == gradProgram3["grad_id"]
person.join(gradProgram3, joinExpr).show()

+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|grad_id| degree|          department|     school|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|      0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|      1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|      1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+



###### How Spark Performs Joins:
To understand how Spark performs joins, we need to understand the two core resources at play:
* node-to-node communication strategy
* per node computation stratergy

Spark approaches cluster communication in two different ways during joins. It either incurs a
* shuffle join, which results in an all-to-all communication. When you join a big table to another big table, you end up with a shuffle join. 
* a broadcast join

The core foundation of simplified view of joins is that in Spark we will have either a **big table** or a **small table.**

**Big table to Big table:**
* when you join big table to big table, we end up in **Shuffle Join**
* In Shuffle join, every node talks to other node and they share data according to which node has a certain key or set of keys(on which you are joining).
* These joins are expensive because the network can be become congested with traffic, especially when the data is not partitioned well.

![Big table to Bigtable](files/tables/Bigtable_to_bigtable-44cdf.PNG)

**Big table–to–small table:**
* This uses Broadcast Join.
* When the table is small enough to fit into the memory of a single worker node, with some breathing room of course, we can optimize our join.
* replicate our small DataFrame onto every worker node in the cluster (be it located on one machine or many).
* Even it looks expensive initially, but this prevents us from performing the all-to-all communication during the entire join process.
* we perform it only once at the beginning and then let each individual worker node perform the work without having to wait or communicate with any other worker node.
* This means that joins will be performed on every single node individually, making CPU the biggest bottleneck.

![Broad Cast Join](files/tables/BroadCast_Join-1b827.PNG)

For our current set of data, we can see that Spark joining type by looking at the explain plan:

In [29]:
joinExpr = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpr).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#10L], [id#24L], Inner
   :- Sort [graduate_program#10L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(graduate_program#10L, 200), ENSURE_REQUIREMENTS, [id=#2381]
   :     +- Project [_1#0L AS id#8L, _2#1 AS name#9, _3#2L AS graduate_program#10L, _4#3 AS spark_status#11]
   :        +- Filter isnotnull(_3#2L)
   :           +- Scan ExistingRDD[_1#0L,_2#1,_3#2L,_4#3]
   +- Sort [id#24L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#24L, 200), ENSURE_REQUIREMENTS, [id=#2382]
         +- Project [_1#16L AS id#24L, _2#17 AS degree#25, _3#18 AS department#26, _4#19 AS school#27]
            +- Filter isnotnull(_1#16L)
               +- Scan ExistingRDD[_1#16L,_2#17,_3#18,_4#19]




With DataFrame API, we can also explicitly give the optimizer a hint that we would like to use a broadcast join by using the correct function around the small DataFrame in question.

In [30]:
from pyspark.sql.functions import broadcast

joinExpr = person["graduate_program"] == graduateProgram["id"]
person.join(broadcast(graduateProgram), joinExpr).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [graduate_program#10L], [id#24L], Inner, BuildRight, false
   :- Project [_1#0L AS id#8L, _2#1 AS name#9, _3#2L AS graduate_program#10L, _4#3 AS spark_status#11]
   :  +- Filter isnotnull(_3#2L)
   :     +- Scan ExistingRDD[_1#0L,_2#1,_3#2L,_4#3]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, true]),false), [id=#2414]
      +- Project [_1#16L AS id#24L, _2#17 AS degree#25, _3#18 AS department#26, _4#19 AS school#27]
         +- Filter isnotnull(_1#16L)
            +- Scan ExistingRDD[_1#16L,_2#17,_3#18,_4#19]




* The SQL interface also includes the ability to provide hints to perform joins. 
* These are not enforced, however, so the optimizer might choose to ignore them.
* We can set one of these hints by using a special comment syntax.
    * MAPJOIN
    * BROADCAST &
    * BROADCASTJOIN 
   
   all do the same thing and are all supported

In [0]:
%sql
SELECT /*+ MAPJOIN(graduateProgram) */ * FROM person JOIN graduateProgram
ON person.graduate_program = graduateProgram.id

id,name,graduate_program,spark_status,id,degree,department,school
0,Bill Chambers,0,List(100),0,Masters,School of Information,UC Berkeley
1,Matei Zaharia,1,"List(500, 250, 100)",1,Ph.D.,EECS,UC Berkeley
2,Michael Armbrust,1,"List(250, 100)",1,Ph.D.,EECS,UC Berkeley


if you try to broadcast something too large, you can crash your driver node (because that collect is expensive).

**Little table–to–little table:**
When performing joins with small tables, it’s usually best to let Spark decide how to join them. You can always force a broadcast join if you’re noticing strange behavior.

###### Conclusion:
It is important to consider is if you **partition** your data correctly prior to a join, you can end up with much more efficient execution because even if a shuffle is planned, if data from two different DataFrames is already located on the same machine, Spark can avoid the
shuffle.

There are additional implications when you decide what order joins should occur in. Because some joins act as filters, this can be a low-hanging improvement in your workloads, as you are guaranteed to reduce data exchanged over the network.